In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import time
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from math import floor, ceil
from pylab import rcParams
import re

%matplotlib inline
engine = sqlite3.connect('DB')

In [2]:
#Write code to convert CSV to Pandas. Pretty sure I lost it somewhere
test = pd.read_csv('./test.csv')
baseTrain = pd.read_csv('./train.csv')
featuresDf = pd.read_csv('./features.csv')

In [3]:
#Combining training data with the features
storesWithTrain = pd.read_sql("""
SELECT *
FROM trainDf
JOIN storesDf 
ON storesDf.Store = trainDf.Store
""", engine)
_, i = np.unique(storesWithTrain.columns, return_index = True)
storesWithTrain = storesWithTrain.iloc[:, i]
storesWithTrain.to_sql('storesWithTrain', engine, if_exists='replace', index = False)

everything = pd.read_sql("""
SELECT *
FROM storesWithTrain
JOIN featuresDf
ON storesWithTrain.Store = featuresDf.Store AND storesWithTrain.Date = featuresDf.Date
""", engine)

#Get all unique elements
_, i = np.unique(everything.columns, return_index = True)
everything = everything.iloc[:, i]

In [4]:
#Grabbing just the basic features that match with the testing data
baseTrain_y = baseTrain['Weekly_Sales']
baseTrain_x = baseTrain.drop('Weekly_Sales', axis = 1)
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
baseTrain_x['Date'] = baseTrain_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 1000000)

In [5]:
#Cleaning the data that has everything

#Parsing the data/convert data time to milliseconds
train_x = everything.drop_duplicates()
train_x = train_x.dropna().reset_index(drop = True)#Get rid of this line if you don't want to drop NaN

#Convert date string to date time object to get milliseconds
train_x['Date'] = train_x['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
train_x['Date'] = train_x['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 1000000)

test['Date'] = test['Date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
test['Date'] = test['Date'].apply(lambda x: int(time.mktime(x.timetuple())) / 1000000)

train_y = train_x['Weekly_Sales']
train_y = train_y.to_frame()
train_y = train_y.dropna().reset_index(drop = True)
train_x.drop('Weekly_Sales', axis = 1, inplace = True)

In [6]:
#Optional cell if you want to remove NaNs. This will mainly remove the MarkDown rows
#One hot code for everything data
storeType = train_x['Type']
encoding = pd.get_dummies(storeType)

train_x['A'] = encoding['A']
train_x['B'] = encoding['B']
train_x['C'] = encoding['C']
train_x.drop('Type', axis = 1, inplace = True)
train_x = train_x.dropna().reset_index(drop = True)

In [7]:
baseTrain_x

,Store,Dept,Date,IsHoliday
0,1,1,1265.3568,False
1,1,1,1265.9616,True
2,1,1,1266.5664,False
3,1,1,1267.1712,False
4,1,1,1267.7760,False
5,1,1,1268.3808,False
6,1,1,1268.9820,False
7,1,1,1269.5868,False
8,1,1,1270.1916,False
9,1,1,1270.7964,False


In [8]:
train_x.to_sql('train_x', engine, if_exists = 'replace', index = False)
train_y.to_sql('train_y', engine, if_exists = 'replace', index = False)
test.to_sql('test', engine, if_exists='replace', index = False)
baseTrain_x.to_sql('baseTrain_x', engine, if_exists='replace', index = False)
baseTrain_y.to_sql('baseTrain_y', engine, if_exists='replace', index = False)